### HHW Lab 2 : GNSS analysis
In this notebook, you'll work through the post-processing steps of GNSS data analysis.

Goals for this notebook:
* Understand the different products available
* Work with GNSS vertical time series
* Evaluate the differences between different GPS stations
* Work through some of the common steps to remove non-hydrologic signals from vertical GNSS timeseries
* Fit an annual model to the signal 
* Remove outliers from the time series 

### Part I : Processed data


For this notebook, you'll be utilizing time series downloaded from the Nevada Geodetic Lab (http://geodesy.unr.edu/) which provides processed GNSS time series. In the previous lectures, we've discussed broadly how GNSS data is processed. Every analysis center processes their data slightly differently and locating the processing strategy documents is important. I would like you to spend a few minutes on the NGL website and find their processing document which explains all the different steps and corrections that are made to the GNSS time series. (Hint: check that right hand panel).

**Q1: Please link the processing strategy document here:**

**Q1.2: In the processing strategy document, what software is used to produce the NGL time series?**

If you're interested in exploring more about the different processing softwares and how they work I highly recommend checking out the websites associated with the different softwares or taking a short course from the groups that maintain and write the programs. In science the most common softwares used are GIPSY and GAMIT, which are also both open source. Some groups also use Bernese, especially European based group, however it is not open source. 

GIPSY link : https://gipsy-oasis.jpl.nasa.gov/

GAMIT link : http://geoweb.mit.edu/gg/ under the workshop tab, unavco also has all their old materials from past workshops. 

UNAVCO has also provided this link which gives the basics in terms of the different softwares: https://www.unavco.org/software/data-processing/postprocessing/postprocessing.html

**Q1.3: You'll see under measurement models, in the processing document, that many different models are corrected for. What ocean tide loading model is used?**

**Q1.4: Is non-tidal atmospheric loading corrected for? What about non-tidal ocean bottom pressure?**

Processing GNSS time series is no small task and proper citation of the analysis center data is crucial whenever using GNSS data. 

**Q2: Please find the preferred citation for the NGL time series and link it here:**


NGL provides multiple different solutions. The first two categorizations are based on the sample rate and the latency (the delay before a transfer of data begins following an instruction for its transfer - aka how long till a data product is available after the data is collected). NGL provides 24 hour sample rate solutions and 5 minute sample rate solutions at different latencies. 

**Q3: 
A final orbit solution has what latency:**

**A rapid orbit solution has what latency:** 

**An ultra rapid solution has what latency:**



Like the name suggests these different solutions have different latencies based on the satellite orbit solution that is being used for the processing. The reason for this is that the orbits are corrected and then all the data is reprocessed with the "final" solution. However, sometimes rapid solutions are needed -- such as after a large earthquake. We will be using the 24 hour final orbit solutions.


NGL additionally provides solutions in multiple different reference frames. Reference frames can be a bit of a rabbit hole -- one that many geodesists venture down. For now, you can think of a reference frame as a point of view. It's indicating how that station is moving relative to a defined point. In the US, the two primary ones are the IGS14 and NA (North American fixed). 

NA fixed reference frame means that its setting a point in the north american tectonic plate as "stable" and rigid which means all motion is relative to that stable point. Most tectonic plates have a reference frame. These reference frames are really useful in tectonic studies becuase you can explore how one plane is moving relative to another.  

IGS14 - is the international GNSS serivce 2014 realization of the International Terrestial Reference Frame 2014 (ITRF14) (older versions are typically labeled based on the year they are created - thus an IGS08 was created in 2008). This is a global reference frame and thus is not tied to any specific tectonic plate. Instead is uses positions from points around the world that are relatively stable (think long time series that are not overly influenced by tides or tectonics) and combines data from GNSS stations and other geodetic measurement systems. 

If you want more information about the IGS reference frame you can nerd out here: https://academic.oup.com/gji/article/209/3/1906/3095992 


Other helpful links for reference frames: https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2014GC005407 or https://www.unavco.org/software/visualization/GPS-Velocity-Viewer/GPS-Velocity-Viewer-frames.html or http://www.nbmg.unr.edu/staff/pdfs/Blewitt_Encyclopedia_of_Geodesy.html

**Q4: Navigate to the map view of stations and click open a station near the west coast of the US. In addition to the IGS and NA reference frames, what reference frame is provided? What tectonic plate is this in reference to?**


We will be using IGS14 solutions. This is a typical choice in hydrogeodesy since we aren't necessarily interested in how one plate is moving relative to another. 

NGL also helpfully provides a list of potential offsets or steps in the time series. Please remember these are *potential* steps. 

**Q5: Please link the step file here:**


NGL also seperates the stepfile into two different step type codes.

**Q6: Code1 is:**

**Q8: Give an example of a type of Code 1:**

**Q7: Code2 is:**

**Q9: What is information is used in Code2:**

**Q10: As if you're writing a paper using this data, please write a sentence describing the GNSS data:**

### Part II : Initial time series

While these first questions and background information may seem a little odd, I want you all to get an understanding of all the different products available so that you can understand what other scientists are using but also so that you can hopefully come back later and find the product you want to use for a specific application. 


Now, that you have some familiarity with the website and the documentation associated with the NGL time series, let's get to work on the actual time series. In the GNSS notebook folder you'll find the time series for multiple GNSS stations. Please import the data (using pandas, please) from CRCN and P571. They are located in the GNSS_data folder so you'll have to include that in the path for importing. Please label the dataframes crcn and p571 and format the date into a datetime object. The units of the position and standard deviations are in mm.

In [1]:
##Import the libraries you'll likely need
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np


Cut both  time series to 1 October 2010 to 1 October 2019 and plot the vertical time series 
I recommend plotting them on separate figures -- remember the command %matplotlib notebook above the plot
will allow for an interactive plot which will be helpful for the following questions. I personally enjoy plotting 
the GNSS time series as a scatter plot which you can do by assigning the marker in the plot command. Just add '.' and
it will plot it as points - the default is lines. I find this easier to diagnose the time series later but it is totally up to you.

**Q10: Please describe the vertical time series of CRCN. Is there anything anomalous in the time series (eg features you might want to remove or be aware of)?**

**Q11: Please describe the vertical time series of P571. Is there anything anomalous in the time series (eg features you might want to remove or be aware of)?**

**Q12: Why do you think these time series look different?**

**Q13: Do you think these two stations are near each other? Ok, now go ahead and look up where these stations are located using the NGL website. Where are the stations located?**

**Q14: Does that change your station to Q12? If yes, any new guesses. Feel free to talk to your neighbors.**

### Part III : Removing non-hydrologic signals

Now, let's start removing some signals from the time series. We'll start by removing some known signals from the time series. Non-tidal atmospheric loading (NTAL) and non-tidal oceanic loading (NTOL) signals are commonly removed to isolate the hydrologic signal in GNSS time series. 

We'll be utilizing the ESMGFZ products (http://esmdata.gfz-potsdam.de:8080/repository/entry/show/Home?entryid=e0fff81f-dcae-469e-8e0a-eb10caf2975b&output=default.html). These are the predicted crustal displacements due to non-tidal atmospheric surface pressure (NTAL) or non-tidal ocean bottom pressure (NTOL). GFZ provides publicly available global grids at sub-daily resolution. If you want more information on how the grids are made check out Dill and Dobslaw, 2013 (https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/jgrb.50353) or the ESMGFZ website linked above. 

More reading on NTAL and NTOL for those interested: https://link.springer.com/article/10.1007/s10291-021-01135-w and https://link.springer.com/article/10.1007/s00190-020-01445-w

The readme files provided by GFZ are really excellent and make using their data easy. However, to save time I've already downloaded the necessary files for the GNSS stations we're using and resampled them to be daily resolution. Go ahead and import them from the group share and then plot them. They are in **meters. Make sure to convert them to mm!**


**Q15: Describe the NTAL and NTOL signals? Similarities? Differences?** 

Since these are elastic displacements, they are additive. The earth is measuring the sum of all the loading signals so we can add the NTOL and NTAL together and then remove it from the GNSS time series. Go ahead and do that now. 

Next, we'll fit the linear trend using a built in python function or by writing your own. A common python function to do this is polyfit (example here: https://www.adamsmith.haus/python/answers/how-to-plot-a-line-of-best-fit-in-python). Just fyi, polyfit does not like NaNs. A good function to remove NaNs from a pandas time series is using df = df.dropna(). 

If you, like me, have set the Date as the index the polyfit function will not work since it does not like dealing with datetime data types. A trick to getting around this while maintaining all the good date information (such as gaps) is by using the datetime function *tooordinal*. Here's an example: https://www.geeksforgeeks.org/python-datetime-toordinal-method-with-example/ But to make it easy on you, here's how I do it. I assign a new column named 't' here (or whatever you like) to be the ordinal. This will convert my dates to a number that can also be mapped back to a date if needed. However, by assigning it to a new column I'll keep the date index so no need for mapping back to a date.  
*df['t']=df.index.map(dt.datetime.toordinal)*

HOWEVER, please rename this linear trend signal something new (eg a new column in the data frame). We'll work with the NTAL and NTOL removed signal again in the next step. Here's one way to do this 

As with everything in python, there are many ways to do the same thing. This is just one option so if you have a different method - go for it! In fact, let me know how you do it. I love learning alternative ways to do things in python. 

When writing new code, I recommend checking the data at each step to make sure its doing what you think it is. It seems time consuming but catching a bug early will save you more time in the long run. Go ahead and plot the linear trend and the vertical data. If it doesn't look right, check your code. 


**Q15: What is the rate of the linear trend in mm/yr for CRCN and for P571?**

**Q16: For CRCN, what do you think the dominate source of this signal is? Feel free to chat with your neighbors.**

**Q17: For P571, what do you think the dominate source of this signal is? Feel free to chat with your neighbors.**

**Q18: For P571 or CRCN, do you think this is the only source of the signal? Can you come up with any other sources that might exhibit similar patterns? Feel free to chat with your neighbors.**

Go ahead and remove the linear trend from the vertical time series. Again, make sure to keep the NTAL/NTOL removed signal as well. And plot the resulting signal. 

**Q19: Describe the resulting signals for CRCN and P571. Are there variations year to year?**

Next, we're going to investigate the seasonal signal. The seasonal signal is typically dominated by hydrologic loading. To estimate the signal we'll work with a portion of the GNSS equation. This will fit a linear trend and a harmonic signal. I've listed the equation below. Those of you who are cruising through this notebook, I highly encourage you to try and code this yourself. Give it a shot! If you get confused you can check out the function I have pre-written. 

We can describe the position of the GNSS vertical timeseries at a given time epoch as:


${y}(t_{i}) = {y}_{0} + {v}({t}_{i}) + {a}\sin(\omega{t}_{i}) + {b}\cos(\omega{t}_{i}) + {c}\sin(2\omega{t}_{i}) + {d}\cos(2\omega{t}_{i})$


Where ${y}_{0}$ is the position at time epoch 0. 

**Q20: And ${v}({t}_{i})$ represents the:**

The last portion of the equation describes the harmonic function. In this case we'll be fitting an annual and semi-annual signal to the time series. $ {a}\sin(\omega{t}_{i}) + {b}\cos(\omega{t}_{i})$ is the annual component, and ${c}\sin(2\omega{t}_{i}) + {d}\cos(2\omega{t}_{i})$ is the semi-annual component. $\omega$ is the period which for annual signals would be $2\pi/365.25$

For those of you using the pre-built function, we'll be solving this equation using least squares. It reads in a columns of a dataframe (eg gps.vertical), the omega, and an ordinal time column (see above if you didn't do it while fitting the linear trend), and a True or False if you want to fit the semi-annual in addition to the annual. It will return the model and the coefficients as an array. Go ahead and fit the model to the vertical component (NTAL and NTOL removed). Don't forget to define the omega. 

*y_v, m = harm_fit(pnum.vertical, omega, pnum.t, True)*

Then m is an array of the coefficients in the harmonic function. So m[0] is y_0 and m[1] is the v(t) and m[2] is a, and so on. 

Plot the vertical time series and the model on the same figure to make sure it looks good. 

**Q21: Looking at your plots of the model and the vertical time series. Does it fit the data perfectly? What is this model actually fitting, eg how are you going to interpret this model?**

Now, lets check out the coefficients. The first coefficient is the position at the start of this time series, the second it the linear trend, the third and fourth can be recombined to determine the annual amplitude and the fifth and sixth can be recombined to determine the semi-annual amplitude. The harmonic coefficients can be negative depending on when in the harmonic cycle the time series starts, so don't worry if they are negative. To determine the amplitude of the harmonic components you'll use the following equation $A = \sqrt{({a}^2 + {b}^2)}$ for the annual signal which means that 

**Q22.1: The semi-annual amplitude would be:** 

Right, now let's calculate them. 

**Q22: What is the amplitude of the annual signal in P571? What about the semi-annual signal?**

**Q24: What is the amplitude of the annual signal in CRCN? What about the semi-annual signal?**

The harmonic also has some timing information. 

**Q: When the harmonic reaches its maximum, what is that telling us? And the minimum?**


Find the min or max of a signal can be done using a handy little function called argmin or argmax: https://numpy.org/doc/stable/reference/generated/numpy.argmax.html 

*variablename = np.argmax(y_v)* or same with argmin

Great, let's figure out when the max and min occurs in both time series.

**Q: P571 reaches its max in what month? And its min?**

**Q: CRCN reaches its max in what month? And its min?**

**Q: Say you were interested in looking at elastic loading across the whole western US or some large area, how might this information be useful?**

Now that we have the harmonics. We're going to removed them from the vertical component (where NTAL and NTOL is removed). Again, just make this a new column in your dataframe. And of course, plot the resulting signal. 

**Q25: What does this resulting signal represent?**

**Q26: For, P571 do you think this model is a good approximation of the annual and semi-annual signal? What about for CRCN?**

**Q27: Let's take a break from the fitting, removing and plotting cycle and scrape the basics of comparisons. Go ahead and check this link about drought in California: https://en.wikipedia.org/wiki/Droughts_in_California#/media/File:Drought_area_in_California.svg. Go ahead and compare the drought metric to the CRCN time series. Describe any patterns and correlations you see.** 

Ok, great break. Now back into it. The next part is a touch harder because I'm not going to give you the code to do this but instead I want to see how all of you might approach this. Use your creativity!

**Q28: You might have noticed that there are outliers in the time series. How might you remove these consistently from the time series? Feel free to chat with your neighbors or chat with us. Remember outliers are data points that differ signigicantly from other nearby observations.**

Let's test it out. Go ahead and code this up and see how it performs. Tinker until it's not removing too much data and instead is just grabbing the outliers. Feel free to chat with your neighbors. If you're really stuck, just ask. But remember there are many ways to do the same thing so use your creativity! 

Alright, speedy coders. Lets dive deeper. We've discussed offsets in the time series and while these time series don't have any huge offsets you can check the UNR website and see that the reason I had you cut the time series to the dates I did was to avoid offsets. While this is one way to avoid offsets in the timeseries, another method is to estimate and correct the time series for the offsets. One way to do that is to include steps in the model we are using. This is typically done by adding a heaviside function in. A heaviside function is a simple step function (https://en.wikipedia.org/wiki/Heaviside_step_function). By doing this we can estimate the amplitude and direction of the offset. 

Here is the new model:

${y}(t_{i}) = {y}_{0} + {v}({t}_{i}) + {a}\sin(\omega{t}_{i}) + {b}\cos(\omega{t}_{i}) + {c}\sin(2\omega{t}_{i}) + {d}\cos(2\omega{t}_{i}) + \sum \limits_{j=1}^{{n}_{g}} {g}_{j} {H}({t}_{i} - {T}_{g})$

where H is the heaviside function, and the magnitude ${g}$ of ${n}_{g}$ offsets occuring at epoch ${T}_{g}$ are due to coseismic or non-coseismic offsets. 

Go ahead and add in the estimation of offsets into your model. Once you think you got your model working, extend your time series for XXX and see how it performs. 

Again, feel free to talk to your neighbors or to us. 


Does your code allow for a variable amount of offsets? Or just a single offset? If just a single offset, see if you can get it to handle a variable amount of offsets. 

A little more? Let's dive into post-seismic signals. After a large earthquake, the elastic deformation that accumulated continues to be released - for long periods after the main shock. This leads to something called post-seismic relaxation or post-seismic decay. If you want to learn more about post-seismic decay check out: https://academic.oup.com/gji/article/155/3/753/629085 or CITATION. 

This relaxation is generally most prominent in the horizontal components but also appears in the vertical component. Fitting the postseismic deformation can be tricky. Generally, it can be seperated into either afterslip (typically modeled with an exponential) or deformation occuring in the mantle (typically modeled with a log). Both are typically added onto the model with a heaviside function -- to account for any co-seismic offset that occurred. 

The exponential function is in the form of: 

$ \sum \limits_{j=1}^{{n}_{h}} {h}_{j} ({1}-{e}^{- \frac{{t}_{i} - {T}_{hj}} {{\tau}_{j}}}) {H}({t}_{i} - {T}_{hj})$

where the post seismic deformation that begins at the time of the earthquake ${T}_{hj}$ and ${\tau}_{j}$ is the time constant which changes based on the earthquake.  

And the log function is in the form of:

$ \sum \limits_{j=1}^{{n}_{k}} {k}_{j} \log ({1}+\frac{{t}_{i} - {T}_{kj}} {{\tau}_{j}}) {H}({t}_{i} - {T}_{kj})$


You can go head and try incorporate the one of these into the model. Again, this is only necessary for large earthquakes. If you want an example of a time series with a large post-seismic deformation you can check out stations AB13, PKDB, HOGS, LAND. 